## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_gkey import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,74.52,83,90,6.91,overcast clouds
1,1,Vaini,TO,-21.2000,-175.2000,78.96,89,75,5.75,broken clouds
2,2,Butaritari,KI,3.0707,172.7902,82.81,70,78,9.75,broken clouds
3,3,Vardo,NO,70.3705,31.1107,37.60,76,100,15.57,light rain
4,4,Emerald,AU,-23.5333,148.1667,74.98,15,0,6.91,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the lowest temperature you would prefer on your trip? "))
max_temp = float(input("What is the highest temperature you would prefer on your trip? "))

What is the lowest temperature you would prefer on your trip? 50
What is the highest temperature you would prefer on your trip? 80


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,74.52,83,90,6.91,overcast clouds
1,1,Vaini,TO,-21.2000,-175.2000,78.96,89,75,5.75,broken clouds
4,4,Emerald,AU,-23.5333,148.1667,74.98,15,0,6.91,clear sky
6,6,Biak,ID,-0.9131,122.8766,79.59,82,84,1.97,broken clouds
8,8,Joshimath,IN,30.5667,79.5667,59.40,27,0,6.89,clear sky
9,9,Atuona,PF,-9.8000,-139.0333,77.85,78,22,17.92,light rain
10,10,Mataura,NZ,-46.1927,168.8643,56.84,46,100,4.97,overcast clouds
13,13,Bluff,NZ,-46.6000,168.3333,54.34,51,93,7.47,overcast clouds
14,14,Bengkulu,ID,-3.8004,102.2655,74.82,85,96,4.21,overcast clouds
16,16,Kuruman,ZA,-27.4524,23.4325,61.00,15,98,10.76,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             368
City                   368
Country                361
Lat                    368
Lng                    368
Max Temp               368
Humidity               368
Cloudiness             368
Wind Speed             368
Current Description    368
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
trip_cities_df = preferred_cities_df.dropna()
len(trip_cities_df)

361

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = trip_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,74.52,overcast clouds,42.0970,-79.2353,
1,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
4,Emerald,AU,74.98,clear sky,-23.5333,148.1667,
6,Biak,ID,79.59,broken clouds,-0.9131,122.8766,
8,Joshimath,IN,59.40,clear sky,30.5667,79.5667,
9,Atuona,PF,77.85,light rain,-9.8000,-139.0333,
10,Mataura,NZ,56.84,overcast clouds,-46.1927,168.8643,
13,Bluff,NZ,54.34,overcast clouds,-46.6000,168.3333,
14,Bengkulu,ID,74.82,overcast clouds,-3.8004,102.2655,
16,Kuruman,ZA,61.00,overcast clouds,-27.4524,23.4325,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    Lat = row["Lat"]
    Long = row["Lng"]
    params["location"] = f"{Lat},{Long}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")
        
        

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_list_df = hotel_df.dropna()
hotel_list_df.count()

City                   361
Country                361
Max Temp               361
Current Description    361
Lat                    361
Lng                    361
Hotel Name             361
dtype: int64

In [24]:
hotel_list_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,74.52,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
1,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
4,Emerald,AU,74.98,clear sky,-23.5333,148.1667,The Irish Village
6,Biak,ID,79.59,broken clouds,-0.9131,122.8766,Penginapan Kim
8,Joshimath,IN,59.40,clear sky,30.5667,79.5667,Shivalik Camps & Resort


In [28]:
# 8a. Create the output File (CSV)
file = open('WeatherPy_vacation.csv', 'w')
# 8b. Export the City_Data into a csv
hotel_list_df.to_csv(file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_list_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_list_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))